In [ ]:
!cp ../input/gdcm-conda-install/gdcm.tar .
!tar -xvzf gdcm.tar
!conda install --offline ./gdcm/gdcm-2.8.9-py37h71b2a6d_0.tar.bz2

In [ ]:
import sys
sys.path.append('../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master')

In [ ]:
import pandas as pd
import numpy as np
import os
import time
import random

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler # for training only, need nightly build pytorch

import pydicom
from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom

from albumentations import Compose, HorizontalFlip, VerticalFlip, RandomRotate90
from albumentations.pytorch import ToTensorV2

In [ ]:
# Configurations
img_inp = {'b0' : 224, 
            'b1' : 240, 
            'b2' : 260, 
            'b3' : 300, 
            'b4' : 380, 
            'b5' : 456, 
            'b6' : 528, 
            'b7' : 600}


pretrained_model = {
    'efficientnet-b0': '../input/efficientnet-pytorch/efficientnet-b0-08094119.pth',
    'efficientnet-b1': '../input/efficientnet-pytorch/efficientnet-b1-dbc7070a.pth', 
    'efficientnet-b2': '../input/efficientnet-pytorch/efficientnet-b2-27687264.pth',
    'efficientnet-b3': '../input/efficientnet-pytorch/efficientnet-b3-c8376fa2.pth', 
    'efficientnet-b4': '../input/efficientnet-pytorch/efficientnet-b4-e116e8b3.pth',
    'efficientnet-b5': '../input/efficientnet-pytorch/efficientnet-b5-586e6cc6.pth', 
    'efficientnet-b6': '../input/efficientnet-pytorch/efficientnet-b6-c76e70fd.pth',
    'efficientnet-b7': '../input/efficientnet-pytorch/efficientnet-b7-dcc49843.pth', 
}


CFG = {
    'train': True,
    
    'train_img_path': '../input/rsna-str-pulmonary-embolism-detection/train',
    'test_img_path': '../input/rsna-str-pulmonary-embolism-detection/test',
    'cv_fold_path': '../input/samplersna/rsna_train_splits_fold_20.csv',
    'train_path': '../input/rsna-str-pulmonary-embolism-detection/train.csv',
    'test_path': '../input/rsna-str-pulmonary-embolism-detection/test.csv',
    
    'image_target_cols': [
        'pe_present_on_image',
         ],
    
    'exam_target_cols': [
        'pe_present_on_image',
        'negative_exam_for_pe', 
        'indeterminate', 
        'both_no', # Added new column
        
        'rv_lv_ratio_gte_1', 
        'rv_lv_ratio_lt_1', 
        
        'chronic_pe', 
        'acute_and_chronic_pe',
        'acute_pe',  # Added new column
        
        'leftsided_pe',
        'central_pe', 
        'rightsided_pe',
        
        'qa_motion',
        'qa_contrast',
        'flow_artifact',
        'true_filling_defect_not_pe'
        ], 
   
    
    'lr': 0.0005,
    'epochs': 1,
    'device': 'cuda', # cuda, cpu
    'train_bs': 64,
    'valid_bs': 64,
    'accum_iter': 1,
    'verbose_step': 1,
    'num_workers': 0,
    'efbnet': 'efficientnet-b3',  # change here
    'img_size': 300,              # change here
    'effnet_fc': 128, 
    'metadata_feats': 26,  
    
    'train_folds': [
                    # [1, 2, 3, 4], 
                    # [0, 2, 3, 4], 
                    # [0, 1, 3, 4], 
                    # [0, 1, 2, 4], 
                    [10, 11, 12, 13]
                   ], 
    
    'valid_folds': [
                    # [0], 
                    # [1], 
                    # [2], 
                    # [3], 
                    [14]
                   ], 
    
    'stage_model_path': '../input/rsna-pre-models/',
    'model_path': '../working/',
    'tag': 'stage1'
}

In [ ]:
# Seed
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
# pre-process train df
def preprocess_DF(df):
    both_no = lambda x: (1 - (x.negative_exam_for_pe + x.indeterminate))
    acute_pe = lambda x: (1 - (x.chronic_pe + x.acute_and_chronic_pe))
    
    df['both_no'] = df.apply(both_no, axis=1)
    df['acute_pe'] = df.apply(acute_pe, axis=1)
    df['acute_pe'] = np.where(df['both_no']==0, 0, df['acute_pe'])
    
    return df

In [ ]:
# Get image + pre-processing

def window_min_max(img, min_, max_, WL=50, WW=350):
    upper, lower = WL+WW//2, WL-WW//2
    X = np.clip(img.copy(), lower, upper)
    X = X - np.min(X)
    X = X / np.max(X)
    return X

def get_img_min_max(path, min_, max_):
    '''
    # min_: patient level pixel min
    # max_: patient level pixel max
    
    RED channel / LUNG window / level=-600, width=1500
    GREEN channel / PE window / level=100, width=700
    BLUE channel / MEDIASTINAL window / level=40, width=400
    '''
    d = pydicom.read_file(path)
    
    # Get image
    img = (d.pixel_array * d.RescaleSlope) + d.RescaleIntercept
    r = window_min_max(img, min_, max_, -600, 1500)
    g = window_min_max(img, min_, max_, 100, 700)
    b = window_min_max(img, min_, max_, 40, 400)
    
    res = np.concatenate([r[:, :, np.newaxis],
                          g[:, :, np.newaxis],
                          b[:, :, np.newaxis]], axis=-1)
    
    res = zoom(res, [CFG['img_size']/res.shape[0], CFG['img_size']/res.shape[1], 1.], prefilter=False, order=1) 
    
    # Get numerical metadata
    SliceThickness           = float(d.SliceThickness)
    KVP                      = float(d.KVP)/100.0
    TableHeight              = float(d.TableHeight)/100.0
    XRayTubeCurrent          = float(d.XRayTubeCurrent)/100.0
    Exposure                 = float(d.Exposure)/100.0
    GantryDetectorTilt       = float(d.GantryDetectorTilt)

    ImagePositionPatient     = [x/100.0 for x in list(d.ImagePositionPatient)]
    ImageOrientationPatient  = list(d.ImageOrientationPatient)
    
    mt_num = np.array((SliceThickness, KVP, TableHeight, 
                XRayTubeCurrent, Exposure, 
                *ImagePositionPatient, *ImageOrientationPatient, 
                GantryDetectorTilt))

    # Get categorical metadata
    SpecificCharacterSet = d.SpecificCharacterSet
    ImageType            = d.ImageType
    ConvolutionKernel    = d.ConvolutionKernel
    PatientPosition      = d.PatientPosition
    
    sps_100     = np.where(SpecificCharacterSet=='ISO_IR 100', 1, 0)
    sps_other   = np.where(sps_100==0, 1, 0)

    it_opa      = np.where(ImageType=="['ORIGINAL', 'PRIMARY', 'AXIAL']", 1, 0)
    it_o        = np.where(ImageType=="ORIGINAL", 1, 0)
    it_other    = np.where(it_opa+it_o > 0, 0, 1)

    ck_std      = np.where(ConvolutionKernel=="STANDARD", 1, 0)
    ck_b        = np.where(ConvolutionKernel=="B", 1, 0)
    ck_other    = np.where(ck_std+ck_b > 0, 0, 1)

    pp_ffs      = np.where(PatientPosition=="FFS", 1, 0)
    pp_hfs      = np.where(PatientPosition=="HFS", 1, 0)
    pp_other    = np.where(pp_ffs+pp_hfs > 0, 0, 1)
    
    mt_cat = np.array((sps_100, sps_other, it_opa, it_o, it_other, ck_std, ck_b, ck_other, pp_ffs, pp_hfs, pp_other))
    
    # Get Metadata
    mt = np.concatenate((mt_num, mt_cat))
    
    return res, mt

In [ ]:
# Dataset

class RSNADataset(TensorDataset):
    def __init__(
        self, df, label_smoothing, data_root, 
        image_subsampling=True, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()        
        self.label_smoothing = label_smoothing
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df[CFG['exam_target_cols']].iloc[index].values
            target[1:-1] = target[0]*target[1:-1]
          
        path = "{}/{}/{}/{}.dcm".format(self.data_root, 
                                        self.df.iloc[index]['StudyInstanceUID'], 
                                        self.df.iloc[index]['SeriesInstanceUID'], 
                                        self.df.iloc[index]['SOPInstanceUID'])
        
        # Get image and metadata
        img, mt  = get_img_min_max(path, 0, 0)
        if self.transforms:
            img = self.transforms(image=img)['image']
        
        # Get metadata and pre-process
        # mt = mt[None, :]
        
        # do label smoothing
        if self.output_label == True:
            target = np.clip(target, self.label_smoothing, 1 - self.label_smoothing)
            
            return img, mt, target
        else:
            return img, mt

In [ ]:
# Image Transformation

def get_train_transforms():
    return Compose([
                    HorizontalFlip(p=0.5),
                    VerticalFlip(p=0.5),
                    RandomRotate90(p=0.5),
                    ToTensorV2(p=1.0),
                    ], p=1.)


def get_valid_transforms():
    return Compose([
                    ToTensorV2(p=1.0),
                    ], p=1.)

In [ ]:
# Models

class FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        # self.cnn_model = EfficientNet.from_pretrained(CFG['efbnet'], in_channels=3)
        self.cnn_model = EfficientNet.from_name(CFG['efbnet'])
        self.cnn_model.load_state_dict(torch.load(pretrained_model[CFG['efbnet']]))
        # self.model._fc = nn.Linear(self.cnn_model._fc.in_features, CFG['effnet_fc'], bias=True)
        self.pooling = nn.AdaptiveAvgPool2d(1)
        
    def get_dim(self):
        return self.cnn_model._fc.in_features
        
    def forward(self, x):
        feats = self.cnn_model.extract_features(x)
        return self.pooling(feats).view(x.shape[0], -1)



class stg1_study_model(nn.Module):
    def __init__(self):
        super().__init__()
        # For image
        self.cnn_model = FeatureExtractor()
        
        # For metadata
        self.fnn_fc1 = nn.Linear(in_features=CFG['metadata_feats'], out_features=32)
        self.fnn_fc2 = nn.Linear(in_features=32, out_features=32)
        self.fnn_fc3 = nn.Linear(in_features=32, out_features=16)
        
        # Final Fusion
        self.final_fc = nn.Linear(in_features=self.cnn_model.get_dim()+16, out_features=len(CFG['exam_target_cols']))
        
    def forward(self, imgs, mts):
        imgs_embdes = self.cnn_model(imgs) # bs * efb_feat_size
        mt_embed = self.fnn_fc1(mts)
        mt_embed = self.fnn_fc2(mt_embed)
        mt_embed = self.fnn_fc3(mt_embed)
        
        embed = torch.cat([imgs_embdes, mt_embed],dim=1)
        
        image_preds = self.final_fc(embed)
        
        return image_preds

In [ ]:
# Loss functions
def rsna_wloss_train(y_true_img, y_pred_img, device):
    bce_func = torch.nn.BCEWithLogitsLoss(reduction='sum').to(device)
    y_pred_img = y_pred_img.view(*y_true_img.shape)
    image_loss = bce_func(y_pred_img, y_true_img)
    correct_count = ((y_pred_img>0) == y_true_img).sum(axis=0)
    counts = y_true_img.size()[0]
    
    return image_loss, correct_count, counts

def rsna_wloss_valid(y_true_img, y_pred_img, device):
    return rsna_wloss_train(y_true_img, y_pred_img, device)

def rsna_wloss_inference(y_true_img, y_pred_img):
    bce_func = torch.nn.BCELoss(reduction='sum')
    image_loss = bce_func(y_pred_img, y_true_img)
    correct_count = ((y_pred_img>0) == y_true_img).sum()
    counts = y_pred_img.shape[0]
    return image_loss, correct_count, counts

In [ ]:
# DataLoader
def prepare_train_dataloader(train, cv_df, train_fold, valid_fold):
    from catalyst.data.sampler import BalanceClassSampler
    
    train_patients = cv_df.loc[cv_df.fold.isin(train_fold), 'StudyInstanceUID'].unique()
    valid_patients = cv_df.loc[cv_df.fold.isin(valid_fold), 'StudyInstanceUID'].unique()

    train_ = train.loc[train.StudyInstanceUID.isin(train_patients),:].reset_index(drop=True)
    valid_ = train.loc[train.StudyInstanceUID.isin(valid_patients),:].reset_index(drop=True)

    # train mode to do image-level subsampling
    train_ds = RSNADataset(train_, 0.0, CFG['train_img_path'],  image_subsampling=False, transforms=get_train_transforms(), output_label=True) 
    valid_ds = RSNADataset(valid_, 0.0, CFG['train_img_path'],  image_subsampling=False, transforms=get_valid_transforms(), output_label=True)

    train_loader = torch.utils.data.DataLoader(
        train_ds,
        batch_size=CFG['train_bs'],
        pin_memory=False,
        drop_last=False,
        shuffle=True,        
        num_workers=CFG['num_workers'],
    )
    
    val_loader = torch.utils.data.DataLoader(
        valid_ds, 
        batch_size=CFG['valid_bs'],
        num_workers=CFG['num_workers'],
        shuffle=False,
        pin_memory=False,
    )
  
    return train_loader, val_loader


In [ ]:
def train_one_epoch(epoch, model, device, scaler, optimizer, train_loader):
    model.train()

    t = time.time()
    loss_sum = 0
    acc_sum = None
    loss_w_sum = 0
    acc_record = []
    loss_record = []
    avg_cnt = 40
    
    for step, (imgs, mts, image_labels) in enumerate(train_loader):
        imgs = imgs.to(device).float()
        mts = mts.to(device).float()
        image_labels = image_labels.to(device).float()

        with autocast():
            image_preds = model(imgs, mts)   #output = model(input)

            image_loss, correct_count, counts = rsna_wloss_train(image_labels, image_preds, device)
            
            loss = image_loss/counts
            scaler.scale(loss).backward()

            loss_ = image_loss.detach().item()/counts
            acc_ = correct_count.detach().cpu().numpy()/counts
            
            loss_record += [loss_]
            acc_record += [acc_]
            loss_record = loss_record[-avg_cnt:]
            acc_record = acc_record[-avg_cnt:]
            loss_sum = np.vstack(loss_record).mean(axis=0)
            acc_sum = np.vstack(acc_record).mean(axis=0)
            
            #loss_w_sum += counts

            if ((step + 1) %  CFG['accum_iter'] == 0) or ((step + 1) == len(train_loader)):
                # may unscale_ here if desired (e.g., to allow clipping unscaled gradients)

                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()                

            acc_details = ["{:.5}: {:.4f}".format(f, float(acc_sum[i])) for i, f in enumerate(CFG['exam_target_cols'])]
            acc_details = ", ".join(acc_details)
            
            if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(train_loader)):
                print(
                    f'epoch {epoch} train Step {step+1}/{len(train_loader)}, ' + \
                    f'loss: {loss_sum[0]:.3f}, ' + \
                    acc_details + ', ' + \
                    f'time: {(time.time() - t):.2f}', end='\r' if (step + 1) != len(train_loader) else '\n'
                )

In [ ]:
def valid_one_epoch(epoch, model, device, scheduler, val_loader, schd_loss_update=False):
    model.eval()

    t = time.time()
    loss_sum = 0
    acc_sum = None
    loss_w_sum = 0

    for step, (imgs, mts, image_labels) in enumerate(val_loader):
        imgs = imgs.to(device).float()
        mts = mts.to(device).float()
        image_labels = image_labels.to(device).float()
        
        image_preds = model(imgs, mts)   #output = model(input)
        #print(image_preds.shape, exam_pred.shape)

        image_loss, correct_count, counts = rsna_wloss_valid(image_labels, image_preds, device)

        loss = image_loss/counts
        
        loss_sum += image_loss.detach().item()
        if acc_sum is None:
            acc_sum = correct_count.detach().cpu().numpy()
        else:
            acc_sum += correct_count.detach().cpu().numpy()
        loss_w_sum += counts     

        acc_details = ["{:.5}: {:.4f}".format(f, acc_sum[i]/loss_w_sum) for i, f in enumerate(CFG['image_target_cols'])]
        acc_details = ", ".join(acc_details)
            
        if ((step + 1) % CFG['verbose_step'] == 0) or ((step + 1) == len(val_loader)):
            print(
                f'epoch {epoch} valid Step {step+1}/{len(val_loader)}, ' + \
                f'loss: {loss_sum/loss_w_sum:.3f}, ' + \
                acc_details + ', ' + \
                f'time: {(time.time() - t):.2f}', end='\r' if (step + 1) != len(val_loader) else '\n'
            )
    
    if schd_loss_update:
        scheduler.step(loss_sum/loss_w_sum)
    else:
        scheduler.step()


## The Actual Run

In [ ]:
seed_everything(SEED)

train_df = pd.read_csv(CFG['train_path'])
cv_df = pd.read_csv(CFG['cv_fold_path'])
train_df = preprocess_DF(train_df)
cv_df = preprocess_DF(cv_df)

print(train_df.shape)
print(cv_df.shape)

In [ ]:
for fold, (train_fold, valid_fold) in enumerate(zip(CFG['train_folds'], CFG['valid_folds'])):
            if fold < 0:
                continue
            
            print(fold)   
            train_loader, val_loader = prepare_train_dataloader(train_df, cv_df, train_fold, valid_fold)

            device = torch.device(CFG['device'])
            model = stg1_study_model().to(device)
            model.load_state_dict(torch.load('{}/model_{}'.format(CFG['stage_model_path'], CFG['tag'])))
            scaler = GradScaler()   
            optimizer = torch.optim.Adam(model.parameters(), lr=CFG['lr'])
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.1, step_size=1); schd_loss_update=False
            
            for epoch in range(CFG['epochs']):
                train_one_epoch(epoch, model, device, scaler, optimizer, train_loader)
                
                torch.save(model.state_dict(),'{}/model_{}'.format(CFG['model_path'], CFG['tag']))
                
                with torch.no_grad():
                    valid_one_epoch(epoch, model, device, scheduler, val_loader, schd_loss_update=schd_loss_update)
            
            torch.save(model.state_dict(),'{}/model_{}'.format(CFG['model_path'], CFG['tag']))
            
            del model, optimizer, train_loader, val_loader, scaler, scheduler
            torch.cuda.empty_cache()